# Day 14| DNN(CNN)

刻一個孿生網路來做數字辨識，什麼是孿生網路？
1. https://zhuanlan.zhihu.com/p/29058453
2. [one-shot learning](https://www.youtube.com/watch?v=r8LLorRACPM)

In [140]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle

In [117]:
### load dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [152]:
### generator

def MNIST_GEN(batch_size, X, y):
    """
    given X and will return the anchor, compare and Y_label
    anchor : size = [None, 28, 28]
    compare : size = [None, 28, 28]
    Y_label : size = [None, 1]
    """
    ### shuffle images and labels ###
    index_anchor = shuffle(range(len(X)))
    index_compare = shuffle(range(len(X)))
    X_anchor = X.take(index_anchor, axis = 0)
    y_anchor = y.take(index_anchor, axis = 0)
    X_compare = X.take(index_compare, axis = 0)
    y_compare = y.take(index_compare, axis = 0)
    
    ### transfrom y, if image of anchor and compare is same, y = 0, else y = 1
    y = (np.equal(y_anchor,y_compare)*-1) + 1
    
    for i in range(0, 60000, batch_size):
        anchor = X_anchor[i:i+batch_size].reshape(batch_size, 28, 28, 1)/ 255.
        compare = X_compare[i:i+batch_size].reshape(batch_size, 28, 28, 1)/ 255.
        Y_label = y[i:i+batch_size].reshape(batch_size, 1)
        yield anchor, compare, Y_label

## Siamese network

#### config

In [103]:
batch_size = 32
epochs = 50
kernel_size = (3,3)
pooling_size = (2,2)
learning_rate = 0.001
margin_siamese = 1

#### graph

In [6]:
tf.reset_default_graph()

In [153]:
siamese_nn = tf.Graph()
with siamese_nn.as_default():

    with tf.name_scope('input'):
        anchor_img = tf.placeholder(dtype = tf.float32, shape = [None, 28, 28, 1], name = 'anchor_image')
        compare_img = tf.placeholder(dtype = tf.float32, shape = [None, 28, 28, 1], name = 'compare_image')
        Y_label = tf.placeholder(dtype = tf.int32, shape = [None, 1], name = 'Y_label')
        learning_rate = tf.placeholder(dtype = tf.float32, shape = [])
        
    with tf.variable_scope('siamese_net'):
        conv_s1_1 = tf.layers.conv2d(anchor_img, filters= 64, kernel_size= kernel_size, activation= tf.nn.relu, name= 'siamese_1')
        conv_s1_1 = tf.layers.max_pooling2d(conv_s1_1, pool_size= pooling_size, strides = (1,1), name= 'siamese_1')
        
        conv_s1_2 = tf.layers.conv2d(conv_s1_1, filters = 32, kernel_size = kernel_size, activation= tf.nn.relu, name= 'siamese_2')
        conv_s1_2 = tf.layers.max_pooling2d(conv_s1_2, pool_size = pooling_size, strides = (1,1), name= 'siamese_2')

        conv_s1_3 = tf.layers.conv2d(conv_s1_2, filters = 16, kernel_size = kernel_size, activation= tf.nn.relu, name= 'siamese_3')
        conv_s1_3 = tf.layers.max_pooling2d(conv_s1_3, pool_size = pooling_size, strides = (1,1), name= 'siamese_3')
        
        anchor_vector = tf.layers.flatten(conv_s1_3, name = 'flatten')
    
    with tf.variable_scope('siamese_net', reuse= True):
        
        conv_s2_1 = tf.layers.conv2d(compare_img, filters= 64, kernel_size= kernel_size, activation= tf.nn.relu, name= 'siamese_1')
        conv_s2_1 = tf.layers.max_pooling2d(conv_s2_1, pool_size= pooling_size, strides = (1,1), name= 'siamese_1')
        
        conv_s2_2 = tf.layers.conv2d(conv_s2_1, filters = 32, kernel_size = kernel_size, activation= tf.nn.relu, name= 'siamese_2')
        conv_s2_2 = tf.layers.max_pooling2d(conv_s2_2, pool_size = pooling_size, strides = (1,1), name= 'siamese_2')

        conv_s2_3 = tf.layers.conv2d(conv_s2_2, filters = 16, kernel_size = kernel_size, activation= tf.nn.relu, name= 'siamese_3')
        conv_s2_3 = tf.layers.max_pooling2d(conv_s2_3, pool_size = pooling_size, strides = (1,1), name= 'siamese_3')
        
        compare_vector = tf.layers.flatten(conv_s2_3, name = 'flatten')
        
    
    with tf.name_scope('loss_function'):
#         loss = tf.contrib.losses.metric_learning.contrastive_loss(Y_label, anchor_vector, compare_vector)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(anchor_vector-compare_vector), 1, keep_dims=True))
        Ls = tf.multiply(0.5, tf.pow(l2_norm,2))
        Ld = tf.multiply(0.5, tf.pow(tf.math.maximum(tf.zeros(1), l2_norm),2))
        loss = tf.reduce_sum(tf.multiply(tf.cast(1-Y_label, tf.float32), Ls) + tf.multiply(tf.cast(Y_label,tf.float32), Ld))
        tf.summary.scalar("Siamese_loss",loss)
        
    with tf.name_scope('training'):
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        optimizer.minimize(loss)
        
    
    merge = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(logdir= './Siamese_train', graph = siamese_nn)
    test_writer = tf.summary.FileWriter(logdir= './Siamese_test', graph = siamese_nn)
    
    init = tf.global_variables_initializer()
    

## Training

In [102]:
from tqdm import tqdm_notebook

In [ ]:
sess = tf.Session(graph= villina_CNN)
sess.run(init)


In [ ]:
epoch_bar = tqdm_notebook(range(epoch))

for epoch in range(epochs):
    
    for 
    